<a href="https://colab.research.google.com/github/JoeHungPham/Python-Project/blob/main/Copy_of_ETL_Master.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import library
import pandas as pd
import numpy as np
import openpyxl
# connect to gg drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Đọc data từ file vào ship và OB cũ
Shipped_old = pd.read_excel("/content/drive/My Drive/RSC/Shipped 290124.xlsx", sheet_name = "Shipped", skiprows = 1)
OB_old = pd.read_excel("/content/drive/My Drive/RSC/Outstanding 290124.xlsx", sheet_name = "Outstanding", skiprows = 1)

In [ ]:
# Đọc data từ file vào ship và OB mới
Shipped_new = pd.read_excel("/content/drive/My Drive/RSC/shipped 02032024.xlsx", sheet_name = "Shipped", skiprows = 1)
OB_new = pd.read_excel("/content/drive/My Drive/RSC/Outstanding020324.xlsx", sheet_name = "Outstanding", skiprows = 1)

In [ ]:
# Đọc các file lookup
RH_lookup = pd.read_excel("/content/drive/My Drive/RSC/Lookup file.xlsx", sheet_name = 'RH lookup')
KA_lookup_finish = pd.read_excel("/content/drive/My Drive/RSC/Lookup file.xlsx", sheet_name = 'KA lookup')
KA_lookup_collection = pd.read_excel("/content/drive/My Drive/RSC/Lookup file.xlsx", sheet_name = 'KA lookup collection')
HS_lookup_collection = pd.read_excel("/content/drive/My Drive/RSC/Lookup file.xlsx", sheet_name = 'HS lookup collection')

In [ ]:
# define hàm để transfrom Shipped + O/B
def Transform(shipped, OB, RH_lookup, KA_lookup_finish, KA_lookup_collection, HS_lookup_collection):
  # Lấy các datafield cần thiết
  shipped = shipped[["CustomerName", "Customer Type", "Collection", "Department",
                     "SO Type", "Item code", "Month Shipped", "Total FOB",
                     "Project Name", "Upholstery Classification"]]

  OB = OB[["CustomerName", "Customer Type",
           "Collection", "Item code", "Department",
           "SO Type", "Current ship month", "FOB Total Value",
           "Upholstery Classification",
           "Project Name",'Final Production Confirm SW']]

  # Đổi tên các cột
  shipped = shipped.rename(columns={'Month Shipped': 'Date', "Total FOB": "Sales"})
  OB = OB.rename(columns={'Current ship month': 'Date', 'FOB Total Value': "Sales"})

  # remove OB azuro
  remove = ['AZZURRO LIVING CO']
  OB = OB[~OB['CustomerName'].isin(remove)]

  # Thêm Order Type cho shipped
  shipped['Order Type'] = 'Shipped'

  # thêm order type cho O/B
  OB_Ordertype = []
  for index, value in OB.iterrows():
    if value['Final Production Confirm SW'] == 'NEW ORDER N/Y CONFIRM SW' or value['Final Production Confirm SW'] is None:
      OB_Ordertype.append('O/B NY confirmed')
    else:
      OB_Ordertype.append('O/B')

  OB['Order Type'] = OB_Ordertype

  OB = OB.drop('Final Production Confirm SW', axis = 1)

  # concat ship và OB lại,
  total = pd.concat([shipped,OB])
  values_to_remove = ['The Mitchell Gold Co']
  total = total[~total['CustomerName'].isin(values_to_remove)]


  # Đổi lại định dạng cột Date
  total['Date'] = pd.to_datetime(total['Date'], format='%Y/M%m')


  # KA
  ###################################
  KA_SO_Type = ['Replacement', 'MTO', 'Swatch', 'SPO', 'Newness', 'Reorder', 'Production - Obsoleted', 'Test-3rd party']
  # lấy Customer Type KA, HS, SD và loại bỏ các SO type không cần thiết
  KA = total[((total["Customer Type"] == "Key Accounts")
            | (total["Customer Type"] == "Sonder Living")
            | (total["Customer Type"] == "Hospitality"))
            & (total["SO Type"].isin(KA_SO_Type))
            ]

  # Biến đổi Newness thành Yes/No và đổi Customer name của khách hàng Sonder Living thành Sonder Living
  Newness = []
  Customer_Collection = []
  for index, value in KA.iterrows():
    if value['Customer Type'] == 'Hospitality' or value['SO Type'] == 'Newness':
      Newness.append('Yes')
    else:
      Newness.append('No')
    if value['Customer Type'] == 'Sonder Living':
      Customer_Collection.append('Sonder Living')
    else: Customer_Collection.append(value['CustomerName'])

  KA['Newness'] = Newness
  KA['CustomerName'] = Customer_Collection

  # Đổi Customer Type của Sonder Living thành Key Accounts
  KA['Customer Type'] = KA['Customer Type'].replace('Sonder Living','Key Accounts')

  # Bỏ những dòng duplicate của KA_lookup_finish do nhập nhầm
  KA_lookup_finish.drop_duplicates(inplace=True)

  # Join KA với KA lookup bằng left join, đổi tên các column
  KA = pd.merge(KA, KA_lookup_finish, how = 'left', on = 'Item code')
  KA = pd.merge(KA, KA_lookup_collection, how = 'left', on = 'CustomerName')
  KA = pd.merge(KA, HS_lookup_collection, how = 'left', on = 'Project Name')

  Customer = []

  for index, value in KA.iterrows():
    if value['Customer Type'] == 'Hospitality':
      Customer.append(value['HS Collection'])
    else: Customer.append(value['Customer/Collection'])

  KA['Customer/Collection'] =  Customer

  KA = KA[['Customer/Collection', 'Customer Type', 'Date', 'Order Type', 'Sales', 'Newness','MAIN CATEGORY']]
  KA = KA.rename(columns={'MAIN CATEGORY': "Finish"})
  KA['Finish'] = KA['Finish'].replace('CASEGOODS (EXCL SG & HG)','CG')
  KA['Finish'] = KA['Finish'].replace('FULL UPHOLSTERY','UPH')
  KA['Customer/Collection'] = KA['Customer/Collection'].astype(str).str.upper()
  KA = KA[KA['Customer/Collection'] != "NAN"]


  # RH

  ############################################
  # Lấy CS Type của RH, loại bỏ các SO Type không cần thiết
  RH_SO_Type = ['Production - Obsoleted', 'SPO',
       'SPO MI/CONTRACT (for contract SPO) - Obsoleted', 'Reorder', 'Newness', 'Replacement', 'DQ','Swatch',
       'SPO/CONTRACT (for contract SPO) - Obsoleted', 'Subcon', 'Contract (for contract Stock) - Obsoleted', 'SPO M/I - Obsoleted']
  RH = total[(total["Customer Type"] == "RH")
           & (total["SO Type"].isin(RH_SO_Type))]


  # Biến đổi RH
  Ordertype = []
  Collection = []
  CustomerType = []
  for index, value in RH.iterrows():
    # shipped -> shipped SPO
    if value['Order Type'] == 'Shipped':
      if value['SO Type'] in ['SPO', 'SPO M/I', 'SPO MI/CONTRACT (for contract SPO)', 'SPO/CONTRACT (for contract SPO)' ]:
        Ordertype.append('Shipped SPO')
      else: Ordertype.append('Shipped')
    else:
      if value['Order Type'] == 'O/B NY confirmed':
        Ordertype.append('O/B NY confirmed')
      else:
        if value['SO Type'] in ['SPO', 'SPO M/I', 'SPO MI/CONTRACT (for contract SPO)', 'SPO/CONTRACT (for contract SPO)' ]:
          Ordertype.append('O/B SPO')
        else: Ordertype.append('O/B')


    # LARA
    if value['SO Type'] == 'Subcon' or value['Department'] == 'LIGHTING':
      Collection.append("Lighting-Ibeam")
    elif value['Collection'] == 'LARA' and value['Department'] == 'BEDROOM':
      Collection.append('LARA BED')

    # NULL -> BCT other collection
    elif pd.isna(value['Collection']):
        if value['Upholstery Classification'] == 'Full Upholstery':
            if value['Department'] == "BCT":
                Collection.append("BCT other UPH Collxn")
            else:
                Collection.append("Main Brand other UPH Collxn")
        else:
            if value['Department'] == "BCT":
                Collection.append("BCT other CG Collxn")
            else:
                Collection.append("Main Brand other CG Collxn")

    # Madero/tomar
    elif value['Collection'] in ["Tomar", "MADERO", "TOMAR"]:
        if value['Department'] == "BCT":
            if  value['Upholstery Classification'] == 'Full Upholstery':
                Collection.append("MADERO/TOMAR UPH BCT")
            else:
                Collection.append("MADERO/TOMAR CG BCT")
        else:
            if  value['Upholstery Classification'] == 'Full Upholstery':
                Collection.append("MADERO/TOMAR UPH CORE")
            else:
                Collection.append("MADERO/TOMAR CG CORE")

    elif value['Upholstery Classification'] == 'Full Upholstery':
        if value["Collection"] == "Aston":
            Collection.append("Aston UPH")
        # elif value["Collection"] == "Maison":
        #     Collection.append("Maison UPH")
        else:
            Collection.append(value["Collection"])
    else: Collection.append(value["Collection"])
  RH['Collection'] = Collection
  RH['Order Type'] = Ordertype

  # uppercase collection, collection lookup
  RH['Collection'] = RH['Collection'].astype(str).str.upper()
  RH_lookup['Collection from shipped'] = RH_lookup['Collection from shipped'].astype(str).str.upper()
  RH_lookup['Collection lookup (review)'] = RH_lookup['Collection lookup (review)'].astype(str).str.upper()

  # Join RH với RH lookup theo collection review
  RH = pd.merge(RH, RH_lookup, how = 'left', left_on = 'Collection', right_on =  'Collection from shipped')
  RH = RH[['Collection lookup (review)', 'Finish', 'Newness', 'Customer Type', 'Order Type', 'Date', 'Sales']]
  RH = RH.rename(columns={'Collection lookup (review)':'Customer/Collection'})

  # Concat RH, KA
  History_data = pd.concat([RH, KA],axis = 0)



  # Biến đổi Date, group lại cho thành 1 tháng
  # History_data['Date'] = pd.to_datetime(History_data['Date'])
  # History_data['Date'] = History_data['Date'].dt.strftime('%m/%Y')
  # History_data['Date'] = pd.to_datetime(History_data['Date'])

  # Loại bỏ các giá trị null trong cột Customer/Collection, Finish, Newwness
  History_data = History_data.dropna(subset=['Customer/Collection', 'Finish', 'Newness'])


  cutoff_date = pd.to_datetime('2023-01-01')
  History_data = History_data[History_data['Date'] >= cutoff_date]


  return History_data

In [ ]:
# biến đổi từ các file Shipped, O/B cũ và mới
History_old = Transform(Shipped_old, OB_old, RH_lookup, KA_lookup_finish, KA_lookup_collection, HS_lookup_collection)
History_new = Transform(Shipped_new, OB_new, RH_lookup, KA_lookup_finish, KA_lookup_collection, HS_lookup_collection)

<ipython-input-5-417106d47804>:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  KA['Newness'] = Newness
<ipython-input-5-417106d47804>:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  KA['CustomerName'] = Customer_Collection
<ipython-input-5-417106d47804>:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexin

In [ ]:
## pivot history cũ và mới
pivot_old = History_old.pivot_table(index =['Customer/Collection', 'Finish', 'Newness', 'Customer Type', 'Order Type'], columns = 'Date', values = 'Sales', aggfunc = 'sum')
pivot_new = History_new.pivot_table(index =['Customer/Collection', 'Finish', 'Newness', 'Customer Type', 'Order Type'], columns = 'Date', values = 'Sales',  aggfunc = 'sum')

In [ ]:
# lấy History_new - History_old
diff_new_old = pivot_new.sub(pivot_old, fill_value = 0)
diff_new_old.reset_index(inplace = True)
# test.fillna(0, inplace=True)

In [ ]:
# Unpivot diff_new_old, đổi order type thành change
dict = {'O/B' : 'O/B Change',
        'Shipped' : 'O/B Change',
        'Shipped SPO' : 'O/B Change',
        'O/B SPO' : 'O/B Change',
        'Shipped Stock' : 'O/B Change',
        'O/B NY confirmed' : 'O/B Change'
        }


diff_new_old['Order Type'] = diff_new_old['Order Type'].replace(dict)


change = diff_new_old.melt(id_vars=['Customer/Collection', 'Finish', 'Newness', 'Customer Type', 'Order Type'], var_name='Date', value_name = 'Sales')

In [ ]:
change = change.replace(0, np.nan)
change = change.dropna(subset=['Sales'])
threshold = 1
change = change[abs(change['Sales']) > threshold]
# change

In [ ]:
# Concat History (bao gồm shipped và O/B hiện tại) và change
total = pd.concat([change,History_new], axis = 0)
total.drop('Newness', axis = 1, inplace = True)

In [ ]:
# Lấy forecast, nor của RH trong file forecast
RH_FC = pd.read_excel("/content/drive/My Drive/RSC/Forecast.xlsx", sheet_name = "RH")
# Front Fill các cột
RH_FC['Finish'].fillna(method = 'ffill', inplace =  True)
RH_FC['Customer/Collection'].fillna(method = 'ffill', inplace =  True)
# RH_FC['Newness'].fillna(method = 'ffill', inplace =  True)
# RH_CMT = RH_FC['Comment']
RH_FC = RH_FC.drop(columns=["Total 2023", "Total 2024", "Total 2025"], axis=1)
try:
  RH_FC = RH_FC.drop(columns=["Comment", "Finish Lookup"], axis=1)
except:
  pass

try:
  RH_FC = RH_FC.drop(columns=["M1 TW", "M2 TW", "M3 TW", "M4 TW", "M5 TW", "M6 TW", "M1 LW", "M2 LW", "M3 LW", "M4 LW",
                            "M5 LW", "M6 LW", "Unnamed: 44", "Unnamed: 45"
                              ],axis=1)
except:
  pass

#  thêm Customer Type
RH_FC['Customer Type'] = 'RH'
RH_FC_melt = RH_FC.melt(id_vars=['Customer/Collection', 'Finish', 'Customer Type', 'Order Type'], var_name='Date', value_name = 'Sales')
# RH_FC_melt.drop('Newness', axis = 1, inplace = True)



value_take = ['Forecast Newness', 'Forecast', 'NOR', 'Forecast SPO', 'Forecast MTO']
RH_FC_melt = RH_FC_melt[RH_FC_melt['Order Type'].isin(value_take)]


In [ ]:
# Lấy forecast, nor của KA trong file forecast
KA_FC = pd.read_excel("/content/drive/My Drive/RSC/Forecast.xlsx", sheet_name = "KA")

# Front Fill các cột
KA_FC['Finish'].fillna(method = 'ffill', inplace =  True)
KA_FC['Customer/Collection'].fillna(method = 'ffill', inplace =  True)
# KA_FC['Newness'].fillna(method = 'ffill', inplace =  True)
# KA_CMT = KA_FC['Comment']
KA_FC = KA_FC.drop(columns=["Total 2023", "Total 2024", "Total 2025"],axis=1)
try:
  KA_FC = KA_FC.drop(columns=["Comment", "Finish Lookup"], axis=1)
except:
  pass
#  thêm Customer Type
KA_FC['Customer Type'] = 'Key Accounts'
KA_FC_melt = KA_FC.melt(id_vars=['Customer/Collection', 'Finish', 'Customer Type', 'Order Type'], var_name='Date', value_name = 'Sales')

value_take = ['Forecast Newness', 'Forecast', 'NOR', 'Forecast SPO', 'Forecast MTO']

# KA_FC_melt.drop('Newness', axis = 1, inplace = True)
KA_FC_melt = KA_FC_melt[KA_FC_melt['Order Type'].isin(value_take)]


In [ ]:
# Lấy forecast, nor của HS trong file forecast
HS_FC = pd.read_excel("/content/drive/My Drive/RSC/Forecast.xlsx", sheet_name = "HS")

# Front Fill các cột
HS_FC['Finish'].fillna(method = 'ffill', inplace =  True)
HS_FC['Customer/Collection'].fillna(method = 'ffill', inplace =  True)
# KA_FC['Newness'].fillna(method = 'ffill', inplace =  True)
# HS_CMT = HS_FC['Comment']
HS_FC = HS_FC.drop(columns=["Total 2023", "Total 2024", "Total 2025"],axis=1)
try:
  HS_FC = HS_FC.drop(columns=["Comment", "Finish Lookup"], axis=1)
except:
  pass

#  thêm Customer Type
HS_FC['Customer Type'] = 'Hospitality'
HS_FC_melt = HS_FC.melt(id_vars=['Customer/Collection', 'Finish', 'Customer Type', 'Order Type'], var_name='Date', value_name = 'Sales')

value_take = ['Forecast Newness', 'Forecast', 'NOR', 'Forecast SPO', 'Forecast MTO']

# KA_FC_melt.drop('Newness', axis = 1, inplace = True)
HS_FC_melt = HS_FC_melt[HS_FC_melt['Order Type'].isin(value_take)]


In [ ]:
# lấy total blank
sum_RH = pd.concat([total, RH_FC_melt], axis = 0)
sum_RH = sum_RH[sum_RH['Customer Type']=='RH']
sum_RH = sum_RH[['Customer/Collection', 'Date', 'Customer Type']]
sum_RH = sum_RH.drop_duplicates()
sum_RH['Sales'] = 0
sum_RH['Finish'] = 'Total'
sum_RH['Order Type'] = 'Total'


In [ ]:
sum_KA = pd.concat([total, KA_FC_melt], axis = 0)
sum_KA = sum_KA[sum_KA['Customer Type']=='Key Accounts']
sum_KA = sum_KA[['Customer/Collection', 'Date', 'Customer Type']]
sum_KA = sum_KA.drop_duplicates()
sum_KA['Sales'] = 0
sum_KA['Finish'] = 'Total'
sum_KA['Order Type'] = 'Total'


In [ ]:


sum_HS = pd.concat([total, HS_FC_melt], axis = 0)
sum_HS = sum_HS[sum_HS['Customer Type']=='Hospitality']
sum_HS = sum_HS[['Customer/Collection', 'Date', 'Customer Type']]
sum_HS = sum_HS.drop_duplicates()
sum_HS['Sales'] = 0
sum_HS['Finish'] = 'Total'
sum_HS['Order Type'] = 'Total'

In [ ]:
# pivot dataframe từ chiều dọc thành chiều ngang để cho vào excel file
# RH

total_RH = pd.concat([total, RH_FC_melt, sum_RH], axis = 0)
# total_RH = total_RH[total_RH['Customer Type']=='RH']
total_RH = total_RH[total_RH['Customer Type']=='RH'].pivot_table(
    index =['Customer/Collection', 'Finish', 'Order Type'], columns = 'Date', values = 'Sales', aggfunc = 'sum')
total_RH = total_RH.replace(0, np.nan)
# total_RH = total_RH.dropna(how='all')

# Sort Order Type
custom_order_type = ['O/B Change',  'Shipped',  'Shipped SPO', 'O/B' , 'O/B SPO',
                'O/B NY confirmed', 'NOR', 'Forecast', 'Forecast Newness','Forecast SPO', 'Forecast MTO', 'Total']

custom_finish = ['CG', 'UPH', 'SHAGREEN', 'HI GLOSS', 'LIGHTING', 'Total']

custom_collection = ['GRAYDON, DELPHINE SHAGREEN', 'ST JAMES', 'MAISON CG + UPH', 'CELA SHAGREEN', 'MAIN BRAND OTHER CG COLLXN', 'BENOIT',
                     'WHITEHALL', 'SUBCON (LAZIO LUCITE & LIGHTING)',
                      'BRETON', 'SANTIAGO',  'CORTA/COLUMN', 'TATE', 'BORA', '2024 MB PLACEHOLDER',  'MAIN BRAND OTHER UPH COLLXN',
                     'MADERO MB UPH + CG',
                     'MADERO BCT UPH + CG', 'BCT LAGUNA', 'BCT OTHER CG COLLXN', 'BCT BELLINA',  'BCT AVALON', 'BCT - CLIO, AUBREY, SHAGREEN',
                     'MODENA', 'ITALIA', 'CONTEMPORARY BEDS','ASTON UPH', '2024 UPH NEWNESS','PARTS & SWATCHES']



sort_order_type = pd.Categorical(total_RH.index.get_level_values('Order Type'), categories=custom_order_type, ordered=True)
sort_finish = pd.Categorical(total_RH.index.get_level_values('Finish'), categories=custom_finish, ordered=True)
sort_collection = pd.Categorical(total_RH.index.get_level_values('Customer/Collection'), categories=custom_collection, ordered=True)

total_RH['sort_order_type'] = sort_order_type
total_RH['sort_finish'] = sort_finish
total_RH['sort_collection'] = sort_collection

total_RH = total_RH.sort_values(by = ['sort_collection', 'sort_finish', 'sort_order_type'], ascending = [True, True, True])
total_RH.drop(columns='sort_order_type', inplace=True)
total_RH.drop(columns='sort_finish', inplace=True)
total_RH.drop(columns='sort_collection', inplace=True)

# total_RH['Comment'] = RH_CMT



In [ ]:

# pivot dataframe từ chiều dọc thành chiều ngang để cho vào excel file
# KA

total_KA = pd.concat([total,KA_FC_melt, sum_KA], axis = 0)
total_KA = total_KA[total_KA['Customer Type']=='Key Accounts'].pivot_table(
    index =['Customer/Collection', 'Finish', 'Order Type'], columns = 'Date', values = 'Sales', aggfunc = 'sum')
total_KA = total_KA.replace(0, np.nan)
# total_KA = total_KA.dropna(how='all')


# Sort Order Type
custom_order_type = ['O/B Change',  'Shipped',  'Shipped SPO', 'O/B' , 'O/B SPO',
                'O/B NY confirmed', 'NOR', 'Forecast', 'Forecast Newness','Forecast SPO', 'Forecast MTO', 'Total']

custom_finish = ['CG', 'UPH', 'SHAGREEN', 'HI GLOSS', 'LIGHTING', 'Total']



sort_order_type = pd.Categorical(total_KA.index.get_level_values('Order Type'), categories=custom_order_type, ordered=True)
sort_finish = pd.Categorical(total_KA.index.get_level_values('Finish'), categories=custom_finish, ordered=True)



total_KA['sort_order_type'] = sort_order_type
total_KA['sort_finish'] = sort_finish

total_KA = total_KA.sort_values(by = ['Customer/Collection', 'sort_finish', 'sort_order_type'], ascending = [True, True, True])
total_KA.drop(columns='sort_order_type', inplace=True)
total_KA.drop(columns='sort_finish', inplace=True)

# total_KA['Comment'] = KA_CMT

In [ ]:
# pivot dataframe từ chiều dọc thành chiều ngang để cho vào excel file
# HS

total_HS = pd.concat([total,HS_FC_melt,sum_HS], axis = 0)
total_HS = total_HS[total_HS['Customer Type']=='Hospitality'].pivot_table(
    index =['Customer/Collection', 'Finish', 'Order Type'], columns = 'Date', values = 'Sales', aggfunc = 'sum')
total_HS = total_HS.replace(0, np.nan)
# total_KA = total_KA.dropna(how='all')


# Sort Order Type
custom_order_type = ['O/B Change',  'Shipped',  'Shipped SPO', 'O/B' , 'O/B SPO',
                'O/B NY confirmed', 'NOR', 'Forecast', 'Forecast Newness','Forecast SPO', 'Forecast MTO', 'Total']

custom_finish = ['CG', 'UPH', 'SHAGREEN', 'HI GLOSS', 'LIGHTING', 'Total']



sort_order_type = pd.Categorical(total_HS.index.get_level_values('Order Type'), categories=custom_order_type, ordered=True)
sort_finish = pd.Categorical(total_HS.index.get_level_values('Finish'), categories=custom_finish, ordered=True)



total_HS['sort_order_type'] = sort_order_type
total_HS['sort_finish'] = sort_finish

total_HS = total_HS.sort_values(by = ['Customer/Collection', 'sort_finish', 'sort_order_type'], ascending = [True, True, True])
total_HS.drop(columns='sort_order_type', inplace=True)
total_HS.drop(columns='sort_finish', inplace=True)

# total_HS['Comment'] = HS_CMT

In [ ]:
# cho vào file excel
excel_file_path = 'Forecast.xlsx'
with pd.ExcelWriter(excel_file_path, engine='openpyxl') as writer:
  total_RH.to_excel(writer, index = True, sheet_name = 'RH')
  total_KA.to_excel(writer, index = True, sheet_name = 'KA')
  total_HS.to_excel(writer, index = True, sheet_name = 'HS')
  # History_new.to_excel(writer, index = True, sheet_name = 'History')

In [ ]:
# Sửa lại các format đúng định dạng
from openpyxl import load_workbook
from openpyxl.styles import PatternFill, Border, Side, Alignment, Protection, Font
from openpyxl.styles import numbers
from openpyxl.styles import NamedStyle
from datetime import datetime
from openpyxl.utils import get_column_letter


file_path = 'Forecast.xlsx'

workbook = load_workbook(file_path)

# lặp qua tất cả các sheet
for sheet_name in workbook.sheetnames:
    ws = workbook[sheet_name]
    alignment = Alignment(horizontal='center', vertical='center')
    # bỏ in đậm chữ và border của tất cả các cells

    # bỏ border, center alignment
    for row in ws.iter_rows(min_row=1, max_row=ws.max_row, min_col=1, max_col=ws.max_column):
        for cell in row:
            # Remove bold formatting for each cell
            cell.border = None
            cell.alignment = alignment

    # bỏ in đậm chữ
    for row in ws.iter_rows(min_row=2, max_row=ws.max_row, min_col=2, max_col=ws.max_column):
        for cell in row:
            # Remove bold formatting for each cell
            cell.font = Font(bold=False)

    # đếm xem trong file có bao nhiêu năm.





    # tính tổng theo column

    for row in ws.iter_rows(min_row=1, min_col=4, max_row=1, max_col=ws.max_column):
      for cell in row:
        if isinstance(cell.value,datetime) and cell.value.month == 12 and cell.value.year == 2023:
          year = cell.value.year
          column_insert = 4
          ws.insert_cols(column_insert)

          column_index = cell.column + 1

          ws.cell(column = column_insert, row=1).value = f"Total {year}"
          ws.cell(column = column_insert, row=1).font = Font(bold=True,color='974706')
          ws.cell(column = column_insert, row=1).fill = PatternFill(start_color="C4BD97", end_color="C4BD97", fill_type="solid")

          start = get_column_letter(column_index-10)
          end = get_column_letter(column_index+1)

          for row_1 in ws.iter_rows(min_row=2, min_col=column_insert, max_row=ws.max_row, max_col=column_insert):
            for cell_1 in row_1:
              cell_row = cell_1.row
              cell_1.value = f"=SUM({start}{cell_row}:{end}{cell_row})"

        ## 2024
        if isinstance(cell.value,datetime) and cell.value.month == 12 and cell.value.year == 2024:
          year = cell.value.year
          column_insert = 5
          ws.insert_cols(column_insert)

          column_index = cell.column + 1

          ws.cell(column = column_insert, row=1).value = f"Total {year}"
          ws.cell(column = column_insert, row=1).font = Font(bold=True,color='974706')
          ws.cell(column = column_insert, row=1).fill = PatternFill(start_color="C4BD97", end_color="C4BD97", fill_type="solid")

          start = get_column_letter(column_index-11)
          end = get_column_letter(column_index)

          for row_1 in ws.iter_rows(min_row=2, min_col=column_insert, max_row=ws.max_row, max_col=column_insert):
            for cell_1 in row_1:
              cell_row = cell_1.row
              cell_1.value = f"=SUM({start}{cell_row}:{end}{cell_row})"

        ## 2025
        if isinstance(cell.value,datetime) and cell.value.month == 12 and cell.value.year == 2025:
          year = cell.value.year
          column_insert = 6
          ws.insert_cols(column_insert)

          column_index = cell.column + 1

          ws.cell(column = column_insert, row=1).value = f"Total {year}"
          ws.cell(column = column_insert, row=1).font = Font(bold=True,color='974706')
          ws.cell(column = column_insert, row=1).fill = PatternFill(start_color="C4BD97", end_color="C4BD97", fill_type="solid")

          start = get_column_letter(column_index-12)
          end = get_column_letter(column_index-1)

          for row_1 in ws.iter_rows(min_row=2, min_col=column_insert, max_row=ws.max_row, max_col=column_insert):
            for cell_1 in row_1:
              cell_row = cell_1.row
              cell_1.value = f"=SUM({start}{cell_row}:{end}{cell_row})"




    # căn lại độ rộng cho mỗi column
    for column_cells in ws.columns:
      length = max(len(str(cell.value)) for cell in column_cells[2:]) + 1
      ws.column_dimensions[column_cells[0].column_letter].width = length



    ####################################
    # tính tổng total line
    sum_values = {'O/B Change', 'Shipped', 'Shipped SPO', 'O/B', 'O/B SPO', 'O/B NY confirmed', 'NOR', 'Forecast', 'Forecast Newness', 'Forecast SPO', 'Forecast MTO'}

    # Tìm vị trí hàng bắt đầu và kết thúc để tính tổng
    start_row = None
    end_row = None

    for row in ws.iter_rows(min_row=2, min_col=3, max_row=ws.max_row, max_col=3):
        cell = row[0]
        if cell.value in sum_values:
            if start_row is None:
                start_row = cell.row
            end_row = cell.row
        elif cell.value == 'Total':
            if start_row is not None and end_row is not None:
                for col in range(4, ws.max_column + 1):
                    col_letter = get_column_letter(col)
                    # total_formula = f"=SUM({col_letter}{start_row}:{col_letter}{end_row})"
                    total_formula = f"=SUMIF(C{start_row}:C{end_row},\"<>O/B Change\",{col_letter}{start_row}:{col_letter}{end_row})"
                    ws.cell(row=cell.row, column=col).value = total_formula
            start_row = None
            end_row = None







    # tô màu chữ theo order type

    order_type_colors = {
    'O/B Change': 'ff0000',
    'O/B': '000000' ,
    'Shipped'  : '00a600',
    'O/B SPO': '000000' ,
    'Shipped SPO'  : '00a600',
    'O/B NY confirmed' : 'f00078',
    'NOR': 'f00078' ,
    'Forecast': '0070C0' ,
    'Forecast Newness': '0070C0',
    'Forecast SPO': '0070C0',
    'Forecast MTO': '0070C0'
    }

    for row in ws.iter_rows(min_row=2, max_row=ws.max_row, min_col=3, max_col=ws.max_column):
      order_type = row[0].value
      if order_type in order_type_colors:
          color = order_type_colors[order_type]
          font = Font(color=color)
          for cell in row:
              cell.font = font

    # tô màu xen kẽ các customer/collection
    fill_gray = PatternFill(start_color="BFBFBF", end_color="BFBFBF", fill_type="solid")
    fill_white = PatternFill(start_color="FFFFFF", end_color="BFBFBF", fill_type="solid")
    last_row = ws.max_row
    cell = ws['A2']
    color = 0
    while cell.row <= last_row:
      if cell.value is not None:
        if color == 0:
          color = 1
          for cells in ws[cell.row]:
            cells.fill = fill_gray
        elif color == 1:
          color = 0
          for cells in ws[cell.row]:
            cells.fill = fill_white
        # cell = ws.cell(row = cell.row + 1, column = cell.column)
      elif cell.value is None:
        if color == 1:
          for cells in ws[cell.row]:
            cells.fill = fill_gray
        else:
          for cells in ws[cell.row]:
            cells.fill = fill_white
      cell = ws.cell(row = cell.row + 1, column = cell.column)

    border_right = Border(right=Side(style='thin'))
    border_top = Border(top=Side(style='thin'))
    border_bottom = Border(bottom=Side(style='thin'))
    border_top_right = Border(top=Side(style='thin'), right=Side(style='thin'))
    border_bottom_right = Border(bottom=Side(style='thin'), right=Side(style='thin'))

    # Lặp qua từng hàng và đặt border bên phải cho từng ô trong cột A, B, C
    for row in ws.iter_rows(min_row=1, min_col=1, max_row=ws.max_row-1, max_col=3):
        for cell in row:
            cell.border = border_right

    for row in ws.iter_rows(min_row=1, min_col=ws.max_column, max_row= ws.max_row, max_col=ws.max_column):
      for cell in row:
        cell.border = border_right

    # Lặp qua từng hàng tô border ở trên
    for row in ws.iter_rows(min_row=1, min_col=1, max_row= ws.max_row, max_col=ws.max_column):
      if row[0].value is not None:  # Kiểm tra ô đầu tiên trong hàng
        for cell in row:
          if cell.column in [1,2,3, ws.max_column]:
            cell.border = border_top_right  # Đặt border cho từng ô
          else:
            cell.border = border_top


    # Lặp qua từng hàng tô border ở trên
    for row in ws.iter_rows(min_row=1, min_col=2, max_row= ws.max_row, max_col=ws.max_column):
      if row[0].value is not None:  # Kiểm tra ô đầu tiên trong hàng
        for cell in row:
          if cell.column in [2,3, ws.max_column]:
            cell.border = border_top_right  # Đặt border cho từng ô
          else:
            cell.border = border_top


    # Áp dụng border cho các ô trong hàng cuối cùng
    for row in ws.iter_rows(min_row=ws.max_row-1, min_col=1, max_row= ws.max_row-1, max_col=ws.max_column):
      for cell in row:
        if cell.column in [1,2,3, ws.max_column]:
          cell.border = border_bottom_right  # Đặt border cho từng ô
        else:
          cell.border = border_bottom

    # đổi lại định dạng ngày tháng ở hàng 1
    for row in ws.iter_rows(min_row=1, min_col=1, max_row=1, max_col=ws.max_column):
        for cell in row:
            # Kiểm tra nếu ô có định dạng là ngày thì định dạng lại thành "mmm-yy"
            if isinstance(cell.value, datetime):
              # cell.value = cell.value.date()
              cell.number_format = 'MMM-YY'

    # đổi định dạng số
    for row in ws.iter_rows(min_row=2, max_row=ws.max_row, min_col=4, max_col=ws.max_column):
        for cell in row:
          cell.number_format = numbers.BUILTIN_FORMATS[3]

    # # căn lại độ rộng cho mỗi column
    # for column_cells in ws.columns:
    #     max_width = 0
    #     for cell in column_cells[2:]:  # Bắt đầu từ hàng thứ 2
    #         if isinstance(cell.value, (int, float)):
    #             cell_length = len(str(cell.value))
    #             if cell_length > max_width:
    #                 max_width = cell_length
    #     if max_width > 0:
    #         ws.column_dimensions[column_cells[0].column_letter].width = max_width + 0.5  # Thêm phần dư để tránh việc bị cắt

    # tô màu cho ô date của tháng hiện tại
    current_month = datetime.now().month
    current_year = datetime.now().year
    for col in range(1, ws.max_column + 1):
      cell = ws.cell(row=1, column=col)
      if isinstance(cell.value, datetime) and cell.value.month == current_month and cell.value.year == current_year:
    # Tô màu cho ô nếu là ngày tháng và tháng bằng với tháng hiện tại
        cell.fill = PatternFill(start_color="2F75B5", end_color="2F75B5", fill_type="solid")
        cell.font = Font(color = "FFFFFF" )


    # tô màu cho cột total
    for col in range(4, ws.max_column + 1):
      cell = ws.cell(row=1, column=col)
      if isinstance(cell.value,str) and 'Total' in cell.value:
        for row in range(1,ws.max_row):
          color_cell = ws.cell(row=row,column=col)
          color_cell.alignment = alignment

    ws.freeze_panes = 'G2'

    ###############################



# Save the modified workbook
workbook.save(file_path)



# format(filepath)


In [ ]:
with pd.ExcelWriter(file_path, engine='openpyxl', mode='a') as writer:
    History_new.to_excel(writer, sheet_name='History', index=False)